In [ ]:
import os

# change the working directory to root of this project
os.chdir("..")

# verify the working directory
os.getcwd()

In [ ]:
import os
import dotenv
import yaml
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

In [ ]:
# load environment variables from .env file
dotenv.load_dotenv(dotenv_path="./config/.env")

# load configuration from config.yaml
with open("./config/config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
username = config["database"]["cassandra"]["username"]
password = os.getenv('CASSANDRA_PASSWORD')
cluster = Cluster(port=29042, auth_provider=PlainTextAuthProvider(username=username, password=password))

In [ ]:
session = cluster.connect()

In [ ]:
sample = session.execute("select release_version from system.local")

In [ ]:
for i in sample:
    print(i)

In [ ]:
from backend.etl.databases.cassandra.cassandra_conn import CassandraConn
from backend.etl.databases.cassandra.table_models import JobListings

conn = CassandraConn()
session = conn.session


In [ ]:
uuids_set = session.execute(
            "SELECT uuid FROM job_listings"
        )
uuids = []
for i in uuids_set:
    uuids.append(str(i['uuid']))

In [ ]:
scrape_dates_set = session.execute(
    "SELECT scraped_at FROM job_listings"
)
scrape_dates = []
for i in scrape_dates_set:
    scrape_dates.append(i['scraped_at'])

In [ ]:
from datetime import datetime

In [ ]:
today = datetime.today() - datetime(2023, 10, 27)
today.days

In [ ]:
jobs_set = session.execute(
    "SELECT * FROM job_listings LIMIT 5"
)
jobs = []
for job in jobs_set:
    jobs.append(job)

In [ ]:
jobs

In [ ]:
full_text = str()

cols = list(jobs[0].keys())[1:]

for col in cols:
    full_text += str(jobs[0][col]) + " "

In [ ]:
full_text

In [ ]:
job = JobListings.objects(uuid=uuids[0]).get()
dict(job)

In [1]:
import os

# change the working directory to root of this project
os.chdir("..")

# verify the working directory
os.getcwd()
from uuid import UUID
from backend.etl.utils.utilities import get_user_metadata, scrub_metadata

In [3]:
meta = get_user_metadata(user_id=UUID("d28a14e5-1970-4e59-9450-6c9bb1dab629"))

In [4]:
meta

'data visualization, machine learning, ABC Inc., Developer, XYZ Corp., Manager, Entry Level, Remote, >200,000, , '

In [2]:
scrub = scrub_metadata(user_id="d28a14e5-1970-4e59-9450-6c9bb1dab629",
                       cuttoff=5)

In [ ]:
{
"user_id": "d28a14e5-1970-4e59-9450-6c9bb1dab629",
"search_id": "6a3a1ec0-c709-4ccc-a71b-eb2c3fc8d745",
"search_timestamp": "2023-11-28T23:37:08.260000",
"search_query": "Data Scientist",
"search_results": [
    "7095930d-e660-aaf0-488b-44aad9338a8a",
    "f7e45f5d-cdc6-4291-dcbc-eef369ccc40a",
    "5e0bd2dd-7306-8066-cbea-dd8a489dd310",
    "3cda4a9d-0724-1957-909d-d67b9c369f78",
    "845aa368-e9e6-c759-eed4-7b4e60d82dc3",
    "8dc57766-e1ed-12e5-f121-4892ba3167f3",
    "134af755-341d-ef66-4f15-f1fee7ec7540",
    "b72f704e-93d6-fdba-3b21-283ef5309864",
    "d7478fb3-17fb-d553-c232-2df85d019a41",
    "c6f9d1a3-b1b1-bd06-98f7-86c26ff67670"
]
}

In [3]:
list(set([1,1,3]))

[1, 3]

In [16]:
from datetime import date

dte = date.today()
str_date = dte.strftime("%Y-%m-%d")
datetime.strptime(str_date, "%Y-%m-%d")
# str_date

datetime.datetime(2023, 12, 2, 0, 0)

In [7]:
import requests
payload = {
    "query": " ",
    "user_id": "72270b81-0790-4e82-9ac3-5ce708e56f98",
}
recommended_jobs = requests.get(
    f"http://localhost:28000/index/search/{payload['user_id']}&&{payload['query']}",
    json=payload
).json()

In [8]:
recommended_jobs

['b3496036-2070-2e44-f567-70898580bad8',
 '538fa3da-fdbc-ecbd-9972-0f012b8efbbf',
 'bfe74f28-11e3-e97c-0e5b-a05fcaa796f3',
 'be057ad5-1011-1354-8d93-1f88111b78b6',
 '9d9ab3d0-6a74-c1ad-cfdb-c2e33b8be3e7',
 'b086deef-268e-1c75-33c3-1bd0cbe5b227',
 'b72f704e-93d6-fdba-3b21-283ef5309864',
 '53d836cd-fb1c-40e2-225d-513360091c22',
 '134af755-341d-ef66-4f15-f1fee7ec7540',
 'e6eed8ec-ab3d-75e3-fb58-ce728d3e5902']

In [7]:
job = requests.get(
    f"http://localhost:28000/jobs/fetch/d7478fb3-17fb-d553-c232-2df85d019a41",
    json={"job_id": "d7478fb3-17fb-d553-c232-2df85d019a41"}
).json()

In [8]:
job

[{'uuid': 'd7478fb3-17fb-d553-c232-2df85d019a41',
  'skipped': False,
  'scraped_at': '2023-11-28T00:36:36.060000',
  'source': 'Jobberman',
  'job_id': 'Not available on Jobberman',
  'job_title': 'Medical Officer',
  'company_name': 'ALLIANCE HOSPITAL AND SERVICES LTD',
  'location': 'Abuja',
  'date': 'Today',
  'job_link': 'https://www.jobberman.com/listings/medical-officer-5d64qn',
  'job_desc': 'Job Summary\n\nWe are looking to hire a suitable candidate to fill this position who will be responsible for providing comprehensive medical care to patients, coordinating with healthcare teams, and ensuring the highest standards of patient safety and satisfaction.\n\nMinimum Qualification: MBBS\nExperience Level: Entry level\nExperience Length: No Experience/Less than 1 year\n\nJob Description/Requirements\nResponsibilities:\nPatient Care:\nConduct medical examinations, assessments, and diagnosis of patients.\nDevelop and implement treatment plans in collaboration with other healthcare p